### **Búsqueda y Minería de Información 2021-22**
### Universidad Autónoma de Madrid, Escuela Politécnica Superior
### Grado en Ingeniería Informática, 4º curso
# **Sistemas de recomendación y análisis de redes sociales**

Fechas:

* Comienzo: lunes 28 / martes 29 de marzo
* Entrega: lunes 9 de mayo, 23:59

# Introducción

## Objetivos

Esta práctica reúne dos objetivos: por una parte implementar y evaluar sistemas de recomendación, y por otra, implementar funcionalidades de análisis de redes sociales. 

Respecto al primer objetivo, se desarrollarán:

* Algoritmos de recomendación basada en filtrado colaborativo.
* Métricas de evaluación de sistemas de recomendación.

Y para el segundo, se implementarán:

* Métricas que se utilizan en el análisis de redes sociales.
* Otras funcionalidades a elección opcional del estudiante, tales como más métricas, la detección de comunidades, la generación aleatoria de redes sociales, o la recomendación de contactos.

## Material proporcionado

Se proporcionan (tanto en el curso de Moodle como con enlaces dentro de este documento) software y datos para la realización de la práctica, que se divide en dos bloques:

**Bloque I – Sistemas de recomendación**

* Un esqueleto de clases y funciones donde el estudiante desarrollará sus implementaciones. De modo similar a las prácticas anteriores, se proporciona una función principal **main_recsys** que deberá funcionar con el código a implementar por el estudiante.
* Dos conjuntos de datos que incluyen ratings asignados por usuarios a películas: un conjunto pequeño de datos ficticios, y otro conjunto de datos reales disponibles en la Web de MovieLens.
  - Conjunto pequeño de prueba <ins>toy-ratings.dat</ins> con datos ficticios de ratings, así como un split manual fijo de estos datos en <ins>toy-train.dat</ins> y <ins>toy-test.dat</ins>.
  - Conjunto de datos real en la Web de [MovieLens](https://grouplens.org/datasets/movielens/latest), disponible en *ml-latest-small.zip*. De los archivos disponibles, se utilizará sólamente <ins>ratings.csv</ins>.
* Un documento de texto <ins>recsys-output.txt</ins> con la correspondiente salida estándar que deberá producir la ejecución de la función **main_recsys**.

**Bloque II – Análisis de redes sociales**

* Un esqueleto de clases y funciones donde el estudiante desarrollará sus implementaciones. De modo similar a las prácticas anteriores, se proporciona una función principal **main_sna** que deberá funcionar con el código a implementar por el estudiante.
* Redes sociales de prueba:
  - Tres redes pequeñas de prueba.
  - Redes reales: la red disponible en [SNAP (facebook_combined)](https://snap.stanford.edu/data/egonets-Facebook.html), y <ins>twitter.csv</ins> obtenida mediante una descarga de Twitter (unos 10 mil usuarios con medio millón de relaciones follow).
  - Al conjunto de redes de prueba, el estudiante añadirá dos redes más, simuladas, en el ejercicio 5.
* Un documento de texto <ins>sna-output.txt</ins> con la correspondiente salida estándar que deberá producir la ejecución de la función **main_sna**.

## Calificación

Esta práctica se calificará con una puntuación de 0 a 10 atendiendo a las puntuaciones individuales de ejercicios y apartados dadas en el enunciado. El peso de la nota de esta práctica en la calificación final de prácticas es del **40%**.

La calificación se basará en el **número** de ejercicios realizados y la **calidad** de los mismos. La puntuación que se indica en cada apartado es orientativa, en principio se aplicará tal cual se refleja pero podrá matizarse por criterios de buen sentido si se da el caso.

Para dar por válida la realización de un ejercicio, el código deberá funcionar (a la primera) integrado con las clases que se facilitan. El profesor comprobará este aspecto combinando las clases entregadas por el estudiante en los programas main facilitados, así como en otros adicionales.

La corrección de las implementaciones se observará por la **coherencia de los resultados** (por ejemplo, las métricas sobre los algoritmos de recomendación), y se valorará la eficiencia en tiempo de ejecución.

## Entrega

La entrega consistirá en un único fichero tipo *notebook* donde se incluirán todas las **implementaciones** solicitadas en cada ejercicio, así como una explicación de cada uno a modo de **memoria**.

## Indicaciones

La realización de los ejercicios conducirá en muchos casos a la implementación de funciones y/o clases adicionales a las que se indican en el enunciado. Algunas vendrán dadas por su aparición en las propias funciones main, y otras por conveniencia a criterio del estudiante.

Igual que en prácticas anteriores, no deberán editarse las funciones **main_recsys** y **main_sna**. Estos programas deberán ejecutar sin errores "a la primera" con el código entregado por el estudiante (naturalmente con salvedad de los ejercicios que no se hayan implementado). 


# Bloque I - Recomendación

Los esqueletos que se proporcionan a continuación son a modo de guía: el estudiante puede modificarlo todo libremente, siempre que la función **main_recsys** funcione correctamente **sin cambios**.

Se implementarán estructuras y diferentes algoritmos para el desarrollo de sistemas de recomendación. 

**Importante**: recordar que no deben recomendarse los ítems que los usuarios ya hayan puntuado.

In [48]:
"""
 Copyright (C) 2022 Pablo Castells, Alejandro Bellogín y Andrés Mena

 Este código se ha implementado para la realización de las prácticas de
 la asignatura "Búsqueda y minería de información" de 4º del Grado en
 Ingeniería Informática, impartido en la Escuela Politécnica Superior de
 la Universidad Autónoma de Madrid. El fin del mismo, así como su uso,
 se ciñe a las actividades docentes de dicha asignatura.
"""

import random
import heapq
from abc import ABC, abstractmethod

class Ranking:
    class ScoredItem:
        """
        Clase utilizada para gestionar las comparaciones que se realizan dentro del heap
        """
        def __init__(self, element):
            self.element = element

        def __lt__(self, other):
            """
            En primer lugar se compara el score. En caso de que sean iguales (mismo score),
            se compara usando el itemid (se colocará más arriba el elemento con un itemid menor).
            """
            return self.element[0] < other.element[0] if self.element[0] != other.element[0] \
                else self.element[1] > other.element[1]

        def __eq__(self, other):
            return self.element == other.element

        def __str__(self):
            return str(self.element)

        def __repr__(self):
            return self.__str__()

    def __init__(self, topn):
        self.heap = []
        self.topn = topn
        self.changed = 0

    def add(self, item, score):
        scored_item = self.ScoredItem((score, item))
        if len(self.heap) < self.topn:
            heapq.heappush(self.heap, scored_item)
            self.changed = 1
        elif scored_item > self.heap[0]:
            heapq.heappop(self.heap)
            heapq.heappush(self.heap, scored_item)
            self.changed = 1

    def __iter__(self):
        self.ranking = []
        if self.changed:
            h = self.heap.copy()
            while h:
                self.ranking.append(heapq.heappop(h).element[::-1])
            self.changed = 0
        return reversed(self.ranking)

    def __repr__(self):
        r = "<"
        for item, score in self:
            r = r + str(item) + ":" + str(score) + " "
        return r[0:-1] + ">"

## Ejercicio 1: Estructuras de datos y recomendación simple (1pt)

Implementar las clases necesarias para manejar **datos de entrada** (ratings) para los algoritmos de recomendación. La funcionalidad se implementará en una clase Ratings, que permitirá leer los datos de un fichero de texto, añadir ratings y acceder a ellos, así como un método que genere dos particiones aleatorias de entrenamiento y test, para evaluar y comparar la efectividad de diferentes algoritmos de recomendación. Nota: podéis asumir que los archivo de ratings no incluyen cabeceras.

La **salida** de un recomendador consistirá en un diccionario con un ránking por usuario. Se facilita una clase Ranking basada en heap, similar a la utilizada para motores de búsqueda en la práctica anterior, para almacenar los ránkings de recomendación; la diferencia es que ahora no se devuelven ránkings en respuesta a consultas, sino un ránking por usuario de forma proactiva y en bloque para todos los usuarios presentes en el conjunto de ratings dado (sin que los usuarios lo "pidan" explícitamente).

Implementar un primer **recomendador simple** por rating promedio en una clase `AverageRecommender`. El recomendador sólo recomendará ítems que tengan un mínimo de ratings, mínimo que se indicará como parámetro en el constructor (con ello se mejora el acierto de la recomendación). Se proporciona una clase `MajorityRecommender` a modo de ejemplo en el que el estudiante podrá basarse. También se proporciona `RandomRecommender`, que se utiliza en ocasiones como referencia en experimentos. 

In [49]:
from collections import OrderedDict
from math import sqrt

class Ratings:
    def __init__(self, file="", delim='\t', lines=None):
        if file != "":
            with open(file, "r") as f:
                lines = f.readlines()
            # Crear un diccionario que, para cada usuario, guarde item y el rating = user[item: rating] #
            # Crear diccionario para encontrar rating en funcion del item (funciones item e item_users) = item[user: rating] #
        self.itemRatings, self.ratings = self.lines_to_dict(lines, delim)

    def lines_to_dict(self, lines, delim='\t'):
        itemRatings = OrderedDict()
        retings = OrderedDict()

        for line in lines:
            columns = line.split(delim)

            user = int(columns[0])
            item = int(columns[1])
            rating = float(columns[2])

            if user not in retings:
                retings[user] = {}
            if item not in retings[user]:
                retings[user][item] = rating

            if item not in itemRatings:
                itemRatings[item] = {}
            if user not in itemRatings[item]:
                itemRatings[item][user] = rating

        return  itemRatings, retings

    def dict_to_lines(self, aux_dict):
        lines = []
        for user in aux_dict:
            for item in aux_dict[user]:
                rating = aux_dict[user][item]
                lines.append("{}\t{}\t{}".format(user, item, rating))
        return lines

    def rate(self, user, item, rating):
        if user not in self.ratings:
            self.ratings[user] = {}

        if item not in self.ratings[user]:
            self.ratings[user][item] = rating

        if item not in self.itemRatings:
            self.itemRatings[item] = {}

        if user not in self.itemRatings[item]:
            self.itemRatings[item][user] = rating

    def rating(self, user, item):
        try:
            return self.ratings[user][item]
        except KeyError:
            return 0

    # Ratio entre 0 y 1
    def random_split(self, ratio, seed=None):
        lines = self.dict_to_lines(self.ratings)

        random.seed(seed)
        random.shuffle(lines)

        split = int(ratio * len(lines))

        train = Ratings(lines=lines[:split])
        test = Ratings(lines=lines[split:])

        return train, test

    def nratings(self):
        n = 0

        for user in self.ratings.keys():
            for item in self.ratings[user]:
                n += 1
        return n

    def users(self):
        res = set()
        for user in self.ratings.keys():
            res.add(user)
        return res

    def items(self):
        res = set()
        for item in self.itemRatings.keys():
            res.add(item)
        return res

    def user_items(self, user):
        try:
            return self.ratings[user]
        except KeyError:
            return []

    def item_users(self, item):
        try:
            return self.itemRatings[item]
        except KeyError:
            return []


In [50]:
class Recommender(ABC):
    def __init__(self, training):
        self.training = training
        self.recommendation = {}

    def __repr__(self):
        return type(self).__name__

    @abstractmethod
    def score(self, user, item):
        """ Completar en otros recomenders """

    def recommend(self, topn):
        for user in self.training.users():
            ranking = Ranking(topn)

            # Solo recomendar items que aun no tenga # 
            for item in self.training.items():
                if item not in self.training.user_items(user):
                    ranking.add(item, self.score(user, item))

            self.recommendation[user] = ranking
        return self.recommendation


class RandomRecommender(Recommender):
    def score(self, user, item):
        return random.random()


class MajorityRecommender(Recommender):
    def __init__(self, ratings, threshold=0):
        super().__init__(ratings)
        self.threshold = threshold

    def score(self, user, item):
        return sum(rating >= self.threshold for user, rating in self.training.item_users(item).items())


class AverageRecommender(Recommender):
    def __init__(self, ratings, minimo=0):
        super().__init__(ratings)
        self.minimo = minimo

    def score(self, user, item):
        """ Al indicarse el minimo por el constructor, se checkea que el numero de rankings llegue a ese minimo.
        Por cada rating en user, se obtiene la suma del mismo y se divide entre el numero total de rankings, obteniendo asi la media """

        ratings = self.training.item_users(item).items()
        if len(ratings) >= self.minimo:
            return sum(rating for usesr, rating in ratings)/len(ratings)
        else:
            return 0

### Explicación/documentación

Hay dos tipos de diccionarios: uno que guarda para cada usuario el rating de un item, es decir, en función del usuario, se guarda el item con el rating que le ha dado; el otro, que guarda para cada item, el usuario que lo ha valorado, por lo tanto de un item puedes sacar los usuarios que lo han valorado.
Para crear el primero, se comprueba que el usuario no está ya en el diccionario y después si el item no ha sido valorado por el usuario se guarda, de tal forma que el usuario ha valorado ese item. Para el segundo, primero se comprueba que el item esté en el diccionario para después guardar el rating dependiendo de el usuario que lo valora.

En la clase Recommender, hay tres formas de recomendar: Random, Majority y Average. El primero devolverá una recomendación aleatoria, el segundo devolverá la suma de los ratings de todos los items que hayan sido valorados por los usuarios; el último, devolverá la media del rating cuyo denominador será el número de items con valoraciones.

## Ejercicio 2: Filtrado colaborativo kNN (3pt)

Implementar dos variantes de filtrado colaborativo mediante vecinos próximos orientado a usuarios:

* Implementar la clase `UserKNNRecommender` para realizar filtrado colaborativo **basado en usuario** por *similitud coseno* (sin normalizar por la suma de similitudes). Se sugiere crear los vecindarios "offline" en el constructor del recomendador. Se recomienda asimismo utilizar la clase `Ranking`, que utiliza un heap de ránking, para construir los vecindarios.
* Implementar una variante normalizada `NormUserKNNRecommender`. De forma similar a la recomendación por rating promedio, el algoritmo exigirá un mínimo de ratings de vecinos para aceptar recomendar un ítem (con ello se mejora el acierto de la recomendación).

In [51]:
from math import pow, sqrt

class UserSimilarity(ABC):
    @ abstractmethod
    def sim(self, user1, user2):
        """ Completado en otros UserSimilarity """


class CosineUserSimilarity(UserSimilarity):
    def __init__(self, train):
        self.train = train

    def sim(self, user1, user2):
        # SUM(items_u1 interseccion items_u2) #
        total_sum = 0
        for i in self.train.user_items(user1).keys():
            if i in self.train.user_items(user2).keys():
                total_sum += self.train.rating(user1, i) * self.train.rating(user2, i)

        # Modulo de user1 #
        m1 = 0
        for i in self.train.user_items(user1).keys():
            m1 += pow(self.train.rating(user1, i), 2)
        m1 = sqrt(m1)

        # Modulo de user2 #
        m2 = 0
        for i in self.train.user_items(user2).keys():
            m2 += pow(self.train.rating(user2, i), 2)
        m2 = sqrt(m2)

        return total_sum / (m1 * m2)


class UserKNNRecommender(Recommender):
    def __init__(self, ratings, sim, k):
        super().__init__(ratings)
        self.sim = sim
        self.k = k
        self.dist = {}

        all_users = set(self.training.users())
        # Crear vecindarios de k vecinos mas cercanos
        for user in self.training.users():
            # Quitamos al propio usuario #
            all_users.discard(user)
            
            # Si el usuario no está en el vecindario, lo introducimos en el vecindario con el ranking que tenga #
            if user not in self.dist:
                self.dist[user] = Ranking(k)

            # Si cualquier otro usuario que pueda ser vecino no está, lo introducimos del mismo modo #
            for other_user in all_users:
                if other_user not in self.dist:
                    self.dist[other_user] = Ranking(k)

                # Caluclamos la similitud de coseno entre usuarios y guardamos la distancia #
                similarity = sim.sim(user, other_user)
                if similarity != 0.0:
                    self.dist[user].add(other_user, similarity)
                    self.dist[other_user].add(user, similarity)

    def score(self, user, item):
        total_sum = 0
        for (other_user, distance) in self.dist[user]:
            other_user_ratings = self.training.ratings[other_user]
            if item in other_user_ratings.keys():
                total_sum += distance * other_user_ratings[item]
        return total_sum


class NormUserKNNRecommender(UserKNNRecommender):
    def __init__(self, ratings, sim, k, min):
        super().__init__(ratings, sim, k)
        self.minimo = min

    def score(self, user, item):
        total_sum = 0
        den = 0
        num_ratings = 0

        """ NormUserKNN tiene en cuenta el numero de vecinos efectivos, por lo tanto el vecino 
        tiene que tener un ranking != 0, el numerador será la distancia del usuario a su vecino por
        su rating mientras que el denominador será la suma de las distancias de los usuarios """
        for (other_user, distance) in self.dist[user]:
            other_user_ratings = self.training.ratings[other_user]
            if item in other_user_ratings.keys():
                num_ratings += 1
                total_sum += distance * other_user_ratings[item]
                den += distance
        if den == 0 or num_ratings < self.minimo:
            return 0
        return total_sum / den

### Explicación/documentación

Para el UserKNNRecommender, se recogen todos los vecinos del usuario al que se hará la recomendación (sin introducir al propio usuario), y se calcula la similitud de cosenos entre usuarios. Para dar el score, solo habrá que iterar entre los items que hayan recibido una valoracion de un vecino del usuario e introducir el algoritmo knn.

Para el NormUserKNNRecommender, habrá que hacer lo mismo excepto que el score no tendrá en cuenta las valoraciones de 0 (ya que no serían vecinos efectivos), y el numero total de valoraciones debe ser mayor a un minimo que se pase por parámetro.

## Ejercicio 3: Ampliación de algoritmos (1pt)

Implementar dos variantes adicionales de los algoritmos de filtrado colaborativo y basados en contenido, tales como:

* Un algoritmo colaborativo por **vecinos próximos orientado a ítem**. 
* **Otras variantes** adicionales a elección del estudiante, por ejemplo: similitud de Pearson, kNN centrado en la media. 

Para probar los métodos deberá completarse la función `student_test_recsys()` del apartado siguiente que ilustre la ejecución de las variantes adicionales.

In [52]:
class ItemNNRecommender(Recommender):
    def __init__(self, ratings, sim, k):
        super().__init__(ratings)
        self.sim = sim
        self.k = k
        self.dist = {}

        elementos = set(self.training.items())
        # Crear vecindarios de k vecinos mas cercanos
        for elemento in self.training.items():
            elementos.discard(elemento)
            if elemento not in self.dist:
                self.dist[elemento] = Ranking(k)
            for otro_elemento in elementos:
                if otro_elemento not in self.dist:
                    self.dist[otro_elemento] = Ranking(k)

                similarity = sim.sim(elemento, otro_elemento)
                if similarity != 0.0:
                    self.dist[elemento].add(otro_elemento, similarity)
                    self.dist[otro_elemento].add(elemento, similarity)

    def score(self, user, item):
        summation = 0
        for (otro_elemento, distance) in self.dist[item]:
            otro_elemento_ratings = self.training.ratings[otro_elemento]
            if user in otro_elemento_ratings.keys():
                summation += distance * otro_elemento_ratings[user]
        return summation

### Explicación/documentación

ItemKNNRecommender sigue los mismos principios que UserKNNRecommender, salvo que en vez de estar orientado a usuarios está orientado a items. En este caso se cogen todos los items, se descarta el propio item, y se calcula la similitud de coseno entre los items vecinos. En cuanto al score, es obtener el rating del item y aplicar el algoritmo.

## Ejercicio 4: Evaluación (1pt)

Se desarrollarán clases que permitan calcular métricas para evaluar y comparar el acierto de los recomendadores: se implementarán **precisión** y **recall**. 

Como resumen de este bloque, se incluirá una *tabla con los valores de las métricas* (dos columnas) más el tiempo de ejecución (una columna más) sobre todos los algoritmos implementados (filas), al menos para el conjunto de datos de <ins>ml-latest-small.zip</ins>.

Opcionalmente, se podrán implementar otras métricas a elección del estudiante (nDCG, etc.), cuya prueba se incluirá en la función `student_test_recsys()`.

In [53]:
import math

class EvaluationMetric(ABC):
    def __init__(self, test, cutoff):
        self.test = test
        self.cutoff = cutoff

    def __repr__(self):
        return type(self).__name__ + ("@" + str(self.cutoff) if self.cutoff != math.inf else "")

    def compute(self, recommendation):
        """ Completar """


class Precision(EvaluationMetric):
    def __init__(self, test, cutoff, threshold):
        self.threshold = threshold
        super().__init__(test, cutoff)

    def compute(self, recommendation):
        ret = 0

        for user in recommendation.keys():
            n = 0
            relevantes = 0
            for rec in recommendation[user]:
                try:
                    if self.test.rating(user, rec[0]) >= self.threshold:
                        relevantes += 1
                except:
                    pass  # Si el usuario no está en test la métrica es 0
                n += 1

                if n == self.cutoff:
                    break

            p = relevantes / n
            ret += p

        return ret / len(recommendation.keys())


class Recall(EvaluationMetric):
    def __init__(self, test, cutoff, threshold):
        self.threshold = threshold
        super().__init__(test, cutoff)

    def compute(self, recommendation):
        ret = 0

        for user in recommendation.keys():
            n = 0
            rel_Devueltos = 0
            rel_Totales = 0

            for rec in recommendation[user]:
                try:
                    if self.test.rating(user, rec[0]) >= self.threshold:
                        rel_Devueltos += 1
                except:
                    pass
                n += 1

                if n == self.cutoff:
                    break
            try:
                for item in self.test.user_items(user):
                    if self.test.rating(user, item) >= self.threshold:
                        rel_Totales += 1
                r = rel_Devueltos / rel_Totales
            except:
                r = 0
                pass

            ret += r

        return ret / len(recommendation.keys())

In [54]:
def student_test_recsys(train, test, k, topn, cutoff):

    recomendadores = [RandomRecommender(train), MajorityRecommender(train, threshold=topn), AverageRecommender(train, minimo=3), UserKNNRecommender(train, CosineUserSimilarity(train), k)]
    metricas = [Precision(test, cutoff=cutoff, threshold=topn), Recall(test, cutoff=cutoff, threshold=topn)]

    for recomendador in recomendadores:
        print("Evaluating", recomendador)
        start = time.process_time()
        recommendation = recomendador.recommend(5)
        print("--> elapsed time:", datetime.timedelta(seconds=round(time.process_time() - start)), "<--")
        for metrica in metricas:
            print("   ", metrica, "=", metrica.compute(recommendation))

### Explicación/documentación

El algoritmo Precision obtiene la interseccion de los documentos relevantes y los documentos que se han obtenido en total y divide el valor entre el numero total de documentos que se han obtenido.

El algoritmo Recall obtiene la interseccion de los documentos relevantes y los documentos obtenidos en total (igual que precision), pero divide el resultado entre el numero de documentos relevantes.


|      | Precision@4 | Recall@4 | Tiempo |
| ----- | --------- | -------- | ----- |
| RandomRecomender | 0.05 | 0.2 | 0:00:00 |
| MajorityRecommender | 0.05 | 0.2 | 0:00:00 |
| AverageRecommender | 0.05 | 0.2 | 0:00:00 |
| UserKNNRecommender | 0.05 | 0.2 | 0:00:00 |


|      | Precision@5 | Recall@5 | Tiempo |
| ----- | --------- | -------- | ----- |
| RandomRecomender | 0.0006557377049180328| 0.0005100182149362477 | 0:00:05 |
| MajorityRecommender | 0.07180327868852462 | 0.08222516076149182 | 0:00:17 |
| AverageRecommender | 0.0 | 0.0 | 0:00:14 |
| UserKNNRecommender | 0.08819672131147552 | 0.11508790797101469 | 0:00:14 |

## Programa de prueba **main_recsys**

Descarga los ficheros del curso de Moodle y coloca sus contenidos en una carpeta **recsys_data** en el mismo directorio que este *notebook*.

In [55]:
import datetime
import time
import itertools

def main_recsys():
    random.seed(42)
    print("=========================\nToy test")
    toy_test("recsys_data/toy", '\t')
    print("=========================\nTesting toy dataset")
    test_dataset("recsys_data/toy-ratings.dat", 1, 2, k=4, min=2, topn=4, cutoff=4)
    print("=========================\nTesting MovieLens \"latest-small\" dataset")
    test_dataset("recsys_data/ratings.csv", 35, 1240, k=10, min=3, topn=5, cutoff=5, delimiter=',')


# First tests on toy dataset, using a pre-constructed data split
def toy_test(dataset, separator='\t'):
    training = Ratings(dataset + "-train.dat", separator)
    test = Ratings(dataset + "-test.dat", separator)
    metrics = [Precision(test, cutoff=4, threshold=4), Recall(test, cutoff=4, threshold=4)]
    evaluate_recommenders(training, metrics, k=4, min=2, topn=4)


# More complete testing on a generic dataset
def test_dataset(ratings_file, user, item, k, min, topn, cutoff, delimiter='\t'):
    ratings = Ratings(ratings_file, delimiter)
    # Test Ratings class on the dataset
    test_data(ratings, user, item)
    # Run some recommenders on the entire rating data as input - no evaluation
    test_recommenders(ratings, k, min, topn)
    # Now produce a rating split to re-run the recommenders on the training data and evaluate them with the test data
    train, test = ratings.random_split(0.8)
    metrics = [Precision(test, cutoff, threshold=4), Recall(test, cutoff, threshold=4)]
    evaluate_recommenders(train, metrics, k, min, 2 * topn)  # Double top n to test a slightly deeper ranking

    # Additional testing?
    student_test_recsys(train, test, k, topn, cutoff)


# Test the rating data handling code (Ratings class)
def test_data(ratings, user, item):
    print("-------------------------\nTesting the data structures")
    print(ratings.nratings(), "ratings by", len(ratings.users()), "users on", len(ratings.items()), "items")
    print("Ratings of user", user, ":", ratings.user_items(user))
    print("Ratings of item", item, ":", ratings.item_users(item))

# Run some recommenders on the some rating data as input - no evaluation
def test_recommenders(ratings, k, min, topn):
    print("-------------------------")
    start = time.process_time()
    test_recommender(RandomRecommender(ratings), topn)
    test_recommender(MajorityRecommender(ratings, threshold=4), topn)
    test_recommender(AverageRecommender(ratings, min), topn)
    timer(start)
    start = time.process_time()
    print("Creating user cosine similarity")
    sim = CosineUserSimilarity(ratings)
    timer(start)
    start = time.process_time()
    print("Creating kNN recommender")
    knn = UserKNNRecommender(ratings, sim, k)
    timer(start)
    start = time.process_time()
    test_recommender(knn, topn)
    timer(start)
    start = time.process_time()
    test_recommender(NormUserKNNRecommender(ratings, sim, k, min), topn)
    timer(start)

# Run one recommender on the some rating data as input - no evaluation
def test_recommender(recommender, topn):
    print("Testing", recommender, "(top", str(topn) + ")")
    recommendation = recommender.recommend(topn)
    for user in itertools.islice(recommendation, 4):
        print("    User", user, "->", recommendation[user])

# Create some recommenders and send them for evaluation for a list of given metrics
def evaluate_recommenders(training, metrics, k, min, topn):
    print("-------------------------")
    start = time.process_time()
    evaluate_recommender(RandomRecommender(training), topn, metrics)
    evaluate_recommender(MajorityRecommender(training, threshold=4), topn, metrics)
    evaluate_recommender(AverageRecommender(training, min), topn, metrics)
    sim = CosineUserSimilarity(training)
    knn = UserKNNRecommender(training, sim, k)
    evaluate_recommender(knn, topn, metrics)
    evaluate_recommender(NormUserKNNRecommender(training, sim, k, min), topn, metrics)

# Run one recommender and evaluate a list of metrics on its output
def evaluate_recommender(recommender, topn, metrics):
    print("Evaluating", recommender)
    recommendation = recommender.recommend(topn)
    for metric in metrics:
        print("   ", metric, "=", metric.compute(recommendation))

def timer(start):
    print("--> elapsed time:", datetime.timedelta(seconds=round(time.process_time() - start)), "<--")

main_recsys()

Toy test
-------------------------
Evaluating RandomRecommender
    Precision@4 = 0.05
    Recall@4 = 0.0
Evaluating MajorityRecommender
    Precision@4 = 0.05
    Recall@4 = 0.0
Evaluating AverageRecommender
    Precision@4 = 0.1
    Recall@4 = 0.0
Evaluating UserKNNRecommender
    Precision@4 = 0.15
    Recall@4 = 0.0
Evaluating NormUserKNNRecommender
    Precision@4 = 0.15
    Recall@4 = 0.0
Testing toy dataset
-------------------------
Testing the data structures
22 ratings by 5 users on 10 items
Ratings of user 1 : {1: 1.0, 5: 5.0, 7: 2.0, 10: 5.0}
Ratings of item 2 : {2: 2.0, 4: 2.0}
-------------------------
Testing RandomRecommender (top 4)
    User 1 -> <8:0.9731157639793706 3:0.8071282732743802 4:0.7297317866938179 2:0.6037260313668911>
    User 2 -> <8:0.8617069003107772 3:0.8294046642529949 5:0.6185197523642461 10:0.577352145256762>
    User 3 -> <2:0.7045718362149235 6:0.28938796360210717 8:0.23279088636103018 4:0.22789827565154686>
    User 4 -> <3:0.6356844442644002 6:0.

### Salida obtenida por el estudiante

#=========================
Toy test
#-------------------------
Evaluating RandomRecommender
    Precision@4 = 0.05
    Recall@4 = 0.2
Evaluating MajorityRecommender
    Precision@4 = 0.05
    Recall@4 = 0.2
Evaluating AverageRecommender
    Precision@4 = 0.1
    Recall@4 = 0.4
Evaluating UserKNNRecommender
    Precision@4 = 0.1
    Recall@4 = 0.4
Evaluating NormUserKNNRecommender
    Precision@4 = 0.15
    Recall@4 = 0.6
#=========================
Testing toy dataset
#-------------------------
Testing the data structures
22 ratings by 5 users on 10 items
Ratings of user 1 : {1: 1.0, 5: 5.0, 7: 2.0, 10: 5.0}
Ratings of item 2 : {2: 2.0, 4: 2.0}
#-------------------------
Testing RandomRecommender (top 4)
    User 1 -> <8:0.9731157639793706 3:0.8071282732743802 4:0.7297317866938179 2:0.6037260313668911>
    User 2 -> <8:0.8617069003107772 3:0.8294046642529949 5:0.6185197523642461 10:0.577352145256762>
    User 3 -> <2:0.7045718362149235 6:0.28938796360210717 8:0.23279088636103018 4:0.22789827565154686>
    User 4 -> <3:0.6356844442644002 6:0.37018096711688264 5:0.36483217897008424 1:0.2779736031100921>
Testing MajorityRecommender (top 4)
    User 1 -> <3:1 4:1 6:1 9:1>
    User 2 -> <5:2 1:1 3:1 10:1>
    User 3 -> <3:1 4:1 6:1 7:1>
    User 4 -> <5:2 1:1 3:1 6:1>
Testing AverageRecommender (top 4)
    User 1 -> <4:4.0 6:4.0 9:2.5 2:2.0>
    User 2 -> <5:4.0 10:3.3333333333333335 1:2.6666666666666665 3:0>
    User 3 -> <4:4.0 6:4.0 7:2.6666666666666665 9:2.5>
    User 4 -> <5:4.0 6:4.0 1:2.6666666666666665 9:2.5>
--> elapsed time: 0:00:00 <--
Creating user cosine similarity
--> elapsed time: 0:00:00 <--
Creating kNN recommender
--> elapsed time: 0:00:00 <--
Testing UserKNNRecommender (top 4)
    User 1 -> <4:2.238184464464379 6:1.9851723076118835 9:1.3156632156329737 3:1.121038479592593>
    User 2 -> <10:2.4731236802019043 5:1.9231236802019038 8:1.5000000000000004 3:1.2666666666666666>
    User 3 -> <6:2.309401076758503 3:1.8475208614068024 9:1.8475208614068024 7:1.6725239222140886>
    User 4 -> <5:2.2316605255328623 6:1.5000000000000004 1:0.908212320458273 9:0.5000000000000001>
--> elapsed time: 0:00:00 <--
Testing NormUserKNNRecommender (top 4)
    User 1 -> <4:4.2592592592592595 6:4.1803278688524586 9:2.7704918032786883 2:2.0>
    User 2 -> <5:3.7613065074434435 10:3.560373755618938 1:2.2386934925565565 3:0>
    User 3 -> <7:1.841113295503649 2:0 3:0 4:0>
    User 4 -> <5:4.696259084270828 1:1.911222747187516 3:0 6:0>
--> elapsed time: 0:00:00 <--
#-------------------------
Evaluating RandomRecommender
    Precision@4 = 0.0
    Recall@4 = 0.0
Evaluating MajorityRecommender
    Precision@4 = 0.0
    Recall@4 = 0.0
Evaluating AverageRecommender
    Precision@4 = 0.0
    Recall@4 = 0.0
Evaluating UserKNNRecommender
    Precision@4 = 0.0
    Recall@4 = 0.0
Evaluating NormUserKNNRecommender
    Precision@4 = 0.0
    Recall@4 = 0.0
Evaluating RandomRecommender
    Precision@4 = 0.0
    Recall@4 = 0.0
Evaluating MajorityRecommender
    Precision@4 = 0.0
    Recall@4 = 0.0
Evaluating AverageRecommender
    Precision@4 = 0.0
    Recall@4 = 0.0
Evaluating UserKNNRecommender
    Precision@4 = 0.0
    Recall@4 = 0.0
#=========================
Testing MovieLens "latest-small" dataset
#-------------------------
Testing the data structures
100836 ratings by 610 users on 9724 items
Ratings of user 35 : {11: 4.0, 21: 5.0, 39: 3.0, 50: 5.0, 60: 5.0, 62: 5.0, 150: 5.0, 185: 5.0, 222: 4.0, 231: 3.0, 235: 4.0, 236: 4.0, 237: 3.0, 252: 4.0, 261: 5.0, 266: 2.0, 300: 4.0, 316: 3.0, 339: 5.0, 342: 4.0, 590: 5.0, 592: 4.0, 595: 3.0}
Ratings of item 1240 : {1: 5.0, 7: 5.0, 15: 4.0, 17: 5.0, 18: 4.0, 19: 4.0, 21: 3.5, 28: 4.5, 30: 3.5, 31: 4.0, 45: 3.0, 57: 4.0, 64: 3.5, 66: 4.0, 68: 4.0, 71: 4.0, 75: 4.0, 78: 5.0, 79: 4.0, 91: 4.5, 95: 5.0, 98: 1.0, 115: 5.0, 125: 4.0, 135: 5.0, 140: 3.0, 149: 4.0, 160: 4.0, 164: 5.0, 166: 3.0, 178: 4.5, 182: 2.0, 186: 4.0, 197: 4.0, 198: 3.0, 199: 5.0, 201: 5.0, 202: 4.0, 212: 4.0, 213: 4.0, 217: 2.0, 219: 4.0, 220: 4.5, 222: 3.5, 223: 3.0, 226: 4.0, 231: 4.0, 232: 4.5, 239: 4.0, 246: 4.5, 249: 4.0, 261: 4.0, 265: 5.0, 266: 4.0, 267: 5.0, 272: 4.0, 274: 4.0, 279: 4.0, 282: 4.5, 288: 3.0, 290: 4.0, 292: 4.0, 298: 4.0, 301: 0.5, 303: 5.0, 304: 5.0, 305: 3.5, 307: 2.5, 312: 4.0, 313: 5.0, 330: 2.5, 332: 4.0, 334: 3.5, 346: 2.5, 354: 4.0, 359: 2.5, 368: 4.0, 370: 4.0, 372: 3.0, 376: 4.0, 380: 5.0, 381: 2.5, 385: 4.0, 387: 4.0, 391: 4.0, 393: 0.5, 407: 3.0, 414: 5.0, 419: 3.5, 425: 3.0, 428: 2.5, 432: 3.5, 434: 3.5, 438: 4.0, 439: 4.0, 448: 3.0, 452: 5.0, 453: 4.0, 462: 4.0, 464: 4.0, 465: 4.0, 469: 4.0, 474: 4.0, 475: 4.5, 477: 4.0, 480: 4.0, 483: 3.5, 489: 3.5, 493: 4.0, 514: 4.0, 520: 4.0, 524: 4.0, 528: 4.0, 532: 4.0, 555: 5.0, 561: 4.0, 562: 5.0, 570: 3.5, 573: 4.5, 577: 4.0, 580: 3.5, 590: 4.5, 594: 5.0, 597: 4.0, 599: 3.5, 600: 3.0, 603: 4.0, 606: 4.0, 607: 5.0, 608: 3.5, 610: 5.0}
#-------------------------
Testing RandomRecommender (top 5)
    User 1 -> <5540:0.9999634808539639 56941:0.9999361757421015 3075:0.9998396704824943 595:0.99972327401748 168026:0.9996158100601125>
    User 2 -> <574:0.9999585260956182 161127:0.9999183408022841 1300:0.9997905695756488 37240:0.9995628342595192 1344:0.9994958997671418>
    User 3 -> <93297:0.9997714606780098 114180:0.9997658441203328 5980:0.9996220968629993 26701:0.9992004177274347 2244:0.999182275498943>
    User 4 -> <2176:0.9999865047833005 4442:0.9999652805471292 172887:0.9997948142608275 61026:0.9997685494474438 110586:0.9996956742304444>
Testing MajorityRecommender (top 5)
    User 1 -> <318:274 858:158 589:150 4993:146 7153:140>
    User 2 -> <356:249 296:244 593:225 2571:222 260:201>
    User 3 -> <318:274 356:249 296:244 593:225 2571:222>
    User 4 -> <318:274 356:249 527:175 110:166 50:163>
Testing AverageRecommender (top 5)
    User 1 -> <6460:4.9 26810:4.833333333333333 115122:4.833333333333333 3224:4.75 7121:4.75>
    User 2 -> <6460:4.9 26810:4.833333333333333 115122:4.833333333333333 3224:4.75 7121:4.75>
    User 3 -> <6460:4.9 26810:4.833333333333333 115122:4.833333333333333 3224:4.75 7121:4.75>
    User 4 -> <6460:4.9 26810:4.833333333333333 115122:4.833333333333333 3224:4.75 7121:4.75>
--> elapsed time: 0:00:33 <--
Creating user cosine similarity
--> elapsed time: 0:00:00 <--
Creating kNN recommender
--> elapsed time: 0:00:23 <--
Testing UserKNNRecommender (top 5)
    User 1 -> <589:14.182298865552864 1200:13.925161854229865 2762:13.422014858479724 1610:13.377867223000493 858:13.134549527466069>
    User 2 -> <2959:7.8886254513902765 356:7.181835552316994 2571:7.106969059865804 33794:6.259884762412396 593:6.2012250262452095>
    User 3 -> <1214:2.7575592777601163 1200:2.2072818953871876 1196:1.946306958245116 2858:1.9253209537004983 1198:1.903532023527017>
    User 4 -> <858:13.097305785186183 1206:12.530262741570782 1193:11.363547457596145 1208:11.299206464803456 1222:11.006092453671277>
--> elapsed time: 0:00:15 <--
Testing NormUserKNNRecommender (top 5)
    User 1 -> <1248:5.000000000000001 3108:5.0 1997:4.999999999999999 858:4.877534867861127 4993:4.875465244842439>
    User 2 -> <48780:4.8418786629483845 4995:4.841592558601045 296:4.68141036849306 1193:4.61652970308254 1732:4.5028052159409615>
    User 3 -> <3578:5.0 1215:4.999999999999999 50:4.88125745298354 47:4.842827449800377 1213:4.812480281563266>
    User 4 -> <1104:5.0 1148:5.0 1256:5.0 1635:5.0 3022:5.0>
--> elapsed time: 0:00:37 <--
#-------------------------
Evaluating RandomRecommender
    Precision@5 = 0.0013114754098360656
    Recall@5 = 0.0004626339052568561
Evaluating MajorityRecommender
    Precision@5 = 0.1557377049180332
    Recall@5 = 0.06751278032799131
Evaluating AverageRecommender
    Precision@5 = 0.0006557377049180328
    Recall@5 = 0.0005702066999287241
Evaluating UserKNNRecommender
    Precision@5 = 0.21278688524590236
    Recall@5 = 0.11251095319177255
Evaluating NormUserKNNRecommender
    Precision@5 = 0.1265573770491808
    Recall@5 = 0.07776971257284841
Evaluating RandomRecommender
    Precision@5 = 0.0003278688524590164
    Recall@5 = 0.000819672131147541
Evaluating MajorityRecommender
    Precision@5 = 0.06885245901639347
    Recall@5 = 0.06931171010040936
Evaluating AverageRecommender
    Precision@5 = 0.0003278688524590164
    Recall@5 = 0.000819672131147541
Evaluating UserKNNRecommender
    Precision@5 = 0.09540983606557385
    Recall@5 = 0.10617945377224677

# Bloque II - Análisis de redes sociales

Los esqueletos que se proporcionan en este apartado son a modo de guía; el estudiante puede modificarlo todo libremente, siempre que la función **main_sna** funcione correctamente **sin cambios**.

Para simplificar, en los ejercicios que siguen supondremos que las redes son no dirigidas.

## Ejercicio 5: Preliminares (2pt)

Generar dos **redes sociales simuladas** siguiendo los modelos de Barabási-Albert y Erdös-Rényi. El tamaño y densidad de los grafos se deja a elección propia. Se puede utilizar para ello cualquier herramienta (como NetworkX ([documentación](https://networkx.org/documentation/stable/tutorial.html#graph-generators-and-graph-operations)), o el entorno interactivo de Gephi), o bien programar implementaciones propias (lo cual también es muy sencillo).

Realizar un análisis básico de la **distribución del grado** en las siete redes sociales de la práctica: small x 3, Facebook, Twitter, Barabási-Albert y Erdös-Rényi. Para cada red:

* Generar una gráfica de distribución del grado (utilizando escala log-log cuando ello sea útil) y comprobar en qué medida se observa una distribución power law.
* Verificar si se observa la paradoja de la amistad (en sus diferentes versiones).

Los resultados de este ejercicio no conllevan entrega de software, sino sólo la documentación de los mismos en el apartado de memoria.

In [56]:
# Si se ha realizado algún código, se puede incluir aquí
# Si no se ha utilizado Python, se puede utilizar el apartado de explicación para describir el proceso y las herramientas utilizadas


#import networkx
#import random
#import numpy
#import matplotlib.pyplot as mplot

"""class Barabasi(ABC):
    def __init__(self, initial_nodes, final_nodes, edges_per_node):
        self.initial_nodes = initial_nodes
        self.final_nodes = final_nodes
        self.edges_per_node = edges_per_node

    def random_contact(self, myGraph):
        contact_nodes = []
        for node in myGraph.nodes():
            grado = myGraph.degree(node)

            node_prob = grado / (2 * len(myGraph.edges()))
            contact_nodes.append(node_prob)

        return numpy.random.choice(myGraph.nodes(), p = node_prob)

    def add_contact(self, myGraph, next_node):
        if len(myGraph.edges()) == 0:
            random_cont = 0
        else:
            random_cont = random_contact(myGraph)

        new_contact = (random_cont, next_node)

        if new_contact in myGraph.edges():
            add_contact(myGraph)

        else:
            myGraph.add_edge(next_node, random_cont)

    def compute(self):
        myGraph = networkx.complete_graph(self.initial_nodes)

        next_node = self.initial_nodes
        diff = 0
        for n in range(self.final_nodes - self.initial_nodes):
            myGraph.add_node(self.initial_nodes + diff)
            diff += 1
            for c in range(0, self.edges_per_node):
                add_contact(myGraph, next_node)
            next_node += 1"""


'"\nimport networkx\nimport random\nimport numpy\nimport matplotlib.pyplot as mplot\n\nclass Barabasi(ABC):\n    def __init__(self, initial_nodes, final_nodes, edges_per_node):\n        self.initial_nodes = initial_nodes\n        self.final_nodes = final_nodes\n        self.edges_per_node = edges_per_node\n\n    def random_contact():\n        contact_nodes = []\n        for node in myGraph.nodes():\n            grado = mygraph.degree(node)\n\n            node_prob = grado / (2 * len(mygraph.edges()))\n            contact_nodes.append(node_prob)\n\n        return numpy.random.choice(myGraph.nodes(), p = node_prob)\n\n    def add_contact(self):\n        if len(myGraph.edges()) == 0:\n            random_cont = 0\n        else:\n            random_cont = random_contact()\n\n        new_contact = (random_cont, next_node)\n\n        if new_contact in myGraph.edges():\n            add_contact()\n\n        else:\n            myGraph.add_edge(next_node, random_cont)\n\n    def compute(self):\n 

### Explicación/documentación/imágenes generadas

En este algoritmo de Barabasi-albert (nodos con preferencias) se establece una serie de nodos iniciales (que son los que estarían en u a habitación) y el número de nodos total que habrá, además del número de conexiones que realiza un nodo una vez entre al grafo (a la habitación).  Al ser un algoritmo preferente, los nodos que estén al principio en el grafo tendrán mayor grado que aquellos que entren después, aunque exite un factor aleatorio a la hora de hacer conexiones (función random_contact) y se irán haciendo vecinos en el grafo (función add_contacts). El algoritmo simplemente hará un loop e irá añadiendo nodos, y será la funcion add_contacts la que determine las conexiones de cada nodo. Si una conexión se repite, se vuelve a llamar a la funcion.

## Ejercicio 6: Métricas (2pt)

Se implementarán las siguientes métricas topológicas *desde cero*:

* Coeficiente de **clustering de un usuario**.
* **Arraigo** de un arco (o de un par de usuarios).
* Coeficiente de **clustering de una red social**.
* Coeficiente de **asortatividad** de grado de una red.

In [57]:
from abc import ABC, abstractmethod
import math
import heapq

class UndirectedSocialNetwork:
    def __init__(self, file, delimiter='\t', parse=0):
        self.net = {}
        self.edges = 0
        f = open(file, "r")
        lines = f.readlines()

        """ Por cada linea conectamos los usuarios depediendo del parser que se introduzca"""
        for line in lines:
            u, v = line.split(delimiter)
            if parse != 0:
                u = parse(u)
                v = parse(v)
            else:
                u = u.rstrip()
                v = v.rstrip()

            if u not in self.net:
                self.net[u] = set()
            if v not in self.net:
                self.net[v] = set()
            if u in self.net[v] and v in self.net[u]:
                continue

            self.net[u].add(v)
            self.net[v].add(u)
            self.edges += 1

        f.close()

    def users(self):
        return list(self.net.keys())

    def contacts(self, user):
        return self.net[user]

    def degree(self, user):
        return len(self.net[user])

    def add_contact(self, u, v):
        if u not in self.net:
            self.net[u] = set()
        if v not in self.net:
            self.sn[v] = set()

        self.net[u].add(v)
        self.net[v].add(u)
        self.edges += 1

    def connected(self, u, v):
        if u in self.net[v] or v in self.net[u]:
            return True
        else:
            return False

    def nedges(self):
        return self.edges

class Metric(ABC):
    def __init__(self):
        pass
        
    def __repr__(self):
        return type(self).__name__ 
        
    @abstractmethod
    def compute_all(self, network):
        """" Compute metric on all users or edges of network """


class LocalMetric(Metric):
    def __init__(self, topn):
        self.topn = topn

    @abstractmethod
    def compute(self, network, element):
        """" Compute metric on one user or edge of network """

class UserClusteringCoefficient(LocalMetric): # or LocalMetric? or other useful intermediate class?
    def compute(self, network, element):
        grado = network.degree(element)
        """Den = numero conexiones posibles total entre vecinos de i"""
        den = (grado * (grado - 1)) / 2
        if den == 0:
            return 0
        num = 0
        """Checkear si ls vecinos estan conectados y sumar si lo estan"""
        vecinos = list(network.contacts(element))
        for i in range(len(vecinos)):
            for j in range(i + 1, len(vecinos)):
                if vecinos[j] in network.contacts(vecinos[i]):
                    num += 1
        return num / den

    def compute_all(self, network):
        """Devolver lo que tiene mas valor en la metrica en un ranking"""
        rank = Ranking(self.topn)
        for u in network.users():
            score = self.compute(network, u)
            rank.add(u, score)
        return rank

    def __repr__(self):
        return type(self).__name__ + ("@" + str(self.topn) if self.topn != math.inf else "")

class AvgUserMetric(Metric): # or LocalMetric? or other useful intermediate class?
    def __init__(self, metric):
        self.metric = metric

    def compute_all(self, network):
        ret = 0
        for user in network.users():
            ret += self.metric.compute(network, user)
        return ret / len(network.users())
    pass

class Embeddedness(LocalMetric): # or LocalMetric? or other useful intermediate class?
    """Calcular los Jaccard"""
    def compute(self, network, element):
        interseccion = (network.contacts(element[0]) - set([element[1]])
                 ) & (network.contacts(element[1]) - set([element[0]]))
        union = (network.contacts(element[0]) - set([element[1]])
                 ) | (network.contacts(element[1]) - set([element[0]]))

        if len(union) == 0:
            return 0

        return len(interseccion) / len(union)

    def compute_all(self, network):
        rank = Ranking(self.topn)
        newSet = set()
        for u in network.users():
            for v in network.users():
                if v in newSet or v == u:
                    continue
                score = self.compute(network, (u, v))
                rank.add((u, v), score)
            newSet.add(u)
        return rank

    def __repr__(self):
        return type(self).__name__ + ("@" + str(self.topn) if self.topn != math.inf else "")

class ClusteringCoefficient(Metric): # or LocalMetric? or other useful intermediate class?
    def compute_all(self, network):
        triangulos = 0
        tripletas = 0
        """Calcular el numero de tripletas y los triangulos que se forman.
        El return sera la division de los triangulos por las tuplas.
        Si se multiplica por 3 el numero de triangulos no da el valor esperado."""
        for u in network.users():
            for v in network.contacts(u):
                for x in network.contacts(v):
                    if x == u:
                        continue
                    tripletas += 1
                    if u in network.contacts(x):
                        triangulos += 1

        return triangulos / tripletas

class Assortativity(Metric): # or LocalMetric? or other useful intermediate class?
    def compute_all(self, network):
        s1 = 0
        s2 = 0
        s3 = 0
        metrica = set()
        """Calcular el valor total de la suma para cada usuario"""
        for u in network.users():
            for v in network.contacts(u):
                if v in metrica:
                    continue
                s1 += network.degree(u) * network.degree(v)
            metrica.add(u)
            s2 += math.pow(network.degree(u), 2)
            s3 += math.pow(network.degree(u), 3)

        num_edges = network.nedges()
        return (4 * num_edges * s1 - math.pow(s2, 2)) / (2 * num_edges * s3 - math.pow(s2, 2))

class Ranking:
    class ScoredUser:
        """
        Clase utilizada para gestionar las comparaciones que se realizan dentro del heap
        """

        def __init__(self, element):
            self.element = element

        def __lt__(self, other):
            """
            En primer lugar se compara el score. En caso de que sean iguales (mismo score),
            se compara usando el userid (se colocará más arriba el elemento con un userid menor).
            """
            return self.element[0] < other.element[0] if self.element[0] != other.element[0] \
                else self.element[1] > other.element[1]

        def __eq__(self, other):
            return self.element == other.element

        def __str__(self):
            return str(self.element)

        def __repr__(self):
            return self.__str__()

    def __init__(self, topn):
        self.heap = []
        self.topn = topn
        self.changed = 0

    def add(self, user, score):
        scored_user = self.ScoredUser((score, user))
        if len(self.heap) < self.topn:
            heapq.heappush(self.heap, scored_user)
            self.changed = 1
        elif scored_user > self.heap[0]:
            heapq.heappop(self.heap)
            heapq.heappush(self.heap, scored_user)
            self.changed = 1

    def __iter__(self):
        if self.changed:
            self.ranking = []
            h = self.heap.copy()
            while h:
                self.ranking.append(heapq.heappop(h).element[::-1])
            self.changed = 0
        return reversed(self.ranking)

    def __repr__(self):
        r = "<"
        for user, score in self:
            r += str(user) + ":" + str(score) + " "
        return r[0:-1] + ">"

### Explicación/documentación

Los algoritmos de métricas sociales se han hecho en función de los algoritmos que había en las diapositivas. Además, se ha creado una clase, ScoredUser, la cual tiene exactamente el mismo funcionamiento que la clase ScoredItem proporcionada al principio salvo que está orientada al usuario.

Además, se documentarán en la memoria los tiempos de ejecución de las métricas en las redes de Facebook y Twitter, en una tabla con la siguiente estructura:

|      | Facebook | Twitter |
| ----- | -------- | ------ | 
| Coef. clustering usuario | 0:00:02 | 0:00:23 |
| Embededness | 0:01:18 | 0:15:12 |
| Coef. clustering global | 0:00:05 | 0:01:26 |
| Asortatividad | 0:00:00 | 0:00:00 |

## Ejercicio 7: Ejercicio libre (1pt)

El estudiante desarrollará uno o varios métodos de análisis de redes a su propia elección. Se sugiere por ejemplo: 

* Implementación de métricas adicionales a elección del estudiante, tales como betweenness, closeness, distancia promedio, diámetro, modularidad, etc., integradas en la misma jerarquía de métricas que el ejercicio anterior.
   - Para la modularidad, los colores de los nodos del grafo small3 sugieren una partición; consultar con el profesor si se desea probar con algún conjunto de datos público más.
   - Si la métrica requiere usar el algoritmo de *shortest paths*, este no hace falta implementarlo, se puede usar el de alguna librería, como el disponible en [networkX](https://networkx.org/documentation/stable/reference/algorithms/shortest_paths.html)
* Detección de comunidades y enlaces débiles.
* Creación de modelos para la generación aleatoria de redes sociales (p.e. amigos de amigos, etc.).
* Recomendación de contactos.

Para este ejercicio deberá completarse la implementación de una función adicional `student_test_sna()` ilustrando la ejecución de las métricas y algoritmos implementados. 

El software que se desarrolle se incluirá en la entrega (en tantas celdas como se consideren oportuno), y se documentarán en la memoria las pruebas realizadas y los resultados obtenidos. En caso de que proceda mostrar figuras de grafos, se sugiere utilizar las facilidades de visualización de la herramienta Gephi.

Este ejercicio se evaluará en base a la cantidad, calidad e interés del trabajo realizado.

In [58]:
# Implementación o implementaciones elegidas

In [59]:
def student_test_sna(data):
    pass

### Explicación/documentación

(por hacer)

## Programa de prueba **main_sna**

Descarga los ficheros del curso de Moodle y coloca sus contenidos en una carpeta **graphs** en el mismo directorio que este *notebook*.

In [60]:
import datetime
import time

def main_sna():
    test_network("graphs/small1.csv", ",", 5, 6, 4, int)
    test_network("graphs/small2.csv", ",", 5, 3, 5, int)
    test_network("graphs/small3.csv", ",", 5, "a", "b")
    test_network("graphs/twitter.csv", ",", 5, "el_pais", "ElviraLindo")
    test_network("graphs/facebook_combined.txt", " ", 5, 9, 3, int)
    # test_network("graphs/barabasi.csv", ",", 5, 1, 2, int) # uno de los grafos creados en Ejercicio 5
    # test_network("graphs/erdos.csv", ",", 5, 1, 2, int) # el otro grafo creado en Ejercicio 5


def test_network(file, delimiter, topn, u, v, parse=0):
    print("==================================================\nTesting " + file + " network")
    network = UndirectedSocialNetwork(file, delimiter=delimiter, parse=parse)
    print(len(network.users()), "users and", network.nedges(), "contact relationships")
    print("User", u, "has", network.degree(u), "contacts")

    # Métricas de usuarios
    print("-------------------------")
    test_metric(UserClusteringCoefficient(topn), network, u)

    # Métricas de arcos
    print("-------------------------")
    test_metric(Embeddedness(topn), network, (u, v))

    # Métricas globales de red
    print("-------------------------")
    test_global_metric(ClusteringCoefficient(), network)
    test_global_metric(AvgUserMetric(UserClusteringCoefficient(topn)), network)
    test_global_metric(Assortativity(), network)

    # Otros tests?
    student_test_sna(network)


def test_metric(metric, network, example):
    start = time.process_time()
    print(metric, ":", metric.compute_all(network))
    print(str(metric) + "(" + str(example) + ") =", metric.compute(network, example))
    timer2(start)


def test_global_metric(metric, network):
    start = time.process_time()
    print(metric, "=", metric.compute_all(network))
    timer2(start)


def timer2(start):
    print("--> elapsed time:", datetime.timedelta(seconds=round(time.process_time() - start)), "<--")

main_sna()

Testing graphs/small1.csv network
12 users and 16 contact relationships
User 6 has 4 contacts
-------------------------
UserClusteringCoefficient@5 : <1:1.0 2:1.0 3:1.0 7:1.0 9:1.0>
UserClusteringCoefficient@5(6) = 0.3333333333333333
--> elapsed time: 0:00:00 <--
-------------------------
Embeddedness@5 : <(3, 2):1.0 (9, 7):1.0 (12, 11):1.0 (1, 10):0.6666666666666666 (6, 8):0.6666666666666666>
Embeddedness@5((6, 4)) = 0.16666666666666666
--> elapsed time: 0:00:00 <--
-------------------------
ClusteringCoefficient = 0.5172413793103449
--> elapsed time: 0:00:00 <--
AvgUserMetric = 0.6666666666666666
--> elapsed time: 0:00:00 <--
Assortativity = -0.24271844660194175
--> elapsed time: 0:00:00 <--
Testing graphs/small2.csv network
9 users and 16 contact relationships
User 3 has 8 contacts
-------------------------
UserClusteringCoefficient@5 : <1:0.6666666666666666 2:0.6666666666666666 4:0.6666666666666666 5:0.6666666666666666 6:0.6666666666666666>
UserClusteringCoefficient@5(3) = 0.285714

### Salida obtenida por el estudiante

==================================================
Testing graphs/small1.csv network
12 users and 16 contact relationships
User 6 has 4 contacts
-------------------------
UserClusteringCoefficient@5 : <1:1.0 2:1.0 3:1.0 7:1.0 9:1.0>
UserClusteringCoefficient@5(6) = 0.3333333333333333
--> elapsed time: 0:00:00 <--
-------------------------
Embeddedness@5 : <(3, 2):1.0 (9, 7):1.0 (12, 11):1.0 (1, 10):0.6666666666666666 (6, 8):0.6666666666666666>
Embeddedness@5((6, 4)) = 0.16666666666666666
--> elapsed time: 0:00:00 <--
-------------------------
ClusteringCoefficient = 0.5172413793103449
--> elapsed time: 0:00:00 <--
AvgUserMetric = 0.6666666666666666
--> elapsed time: 0:00:00 <--
Assortativity = -0.24271844660194175
--> elapsed time: 0:00:00 <--
==================================================
Testing graphs/small2.csv network
9 users and 16 contact relationships
User 3 has 8 contacts
-------------------------
UserClusteringCoefficient@5 : <1:0.6666666666666666 2:0.6666666666666666 4:0.6666666666666666 5:0.6666666666666666 6:0.6666666666666666>
UserClusteringCoefficient@5(3) = 0.2857142857142857
--> elapsed time: 0:00:00 <--
-------------------------
Embeddedness@5 : <(1, 4):0.5 (1, 8):0.5 (2, 5):0.5 (4, 6):0.5 (6, 8):0.5>
Embeddedness@5((3, 5)) = 0.2857142857142857
--> elapsed time: 0:00:00 <--
-------------------------
ClusteringCoefficient = 0.46153846153846156
--> elapsed time: 0:00:00 <--
AvgUserMetric = 0.6243386243386244
--> elapsed time: 0:00:00 <--
Assortativity = -0.3333333333333333
--> elapsed time: 0:00:00 <--
==================================================
Testing graphs/small3.csv network
4 users and 5 contact relationships
User a has 3 contacts
-------------------------
UserClusteringCoefficient@5 : <b:1.0 d:1.0 a:0.6666666666666666 c:0.6666666666666666>
UserClusteringCoefficient@5(a) = 0.6666666666666666
--> elapsed time: 0:00:00 <--
-------------------------
Embeddedness@5 : <('a', 'c'):1.0 ('b', 'd'):1.0 ('a', 'b'):0.5 ('a', 'd'):0.5 ('b', 'c'):0.5>
Embeddedness@5(('a', 'b')) = 0.5
--> elapsed time: 0:00:00 <--
-------------------------
ClusteringCoefficient = 0.75
--> elapsed time: 0:00:00 <--
AvgUserMetric = 0.8333333333333333
--> elapsed time: 0:00:00 <--
Assortativity = -0.6666666666666666
--> elapsed time: 0:00:00 <--
==================================================
Testing graphs/twitter.csv network
10029 users and 462399 contact relationships
User el_pais has 1899 contacts
-------------------------
UserClusteringCoefficient@5 : <AlanaMurrin:1.0 AsFerreiro:1.0 AsTheyBurn:1.0 BJRatedR:1.0 BarrierMetal:1.0>
UserClusteringCoefficient@5(el_pais) = 0.04107979852964596
--> elapsed time: 0:00:23 <--
-------------------------
Embeddedness@5 : <('AffiliateMoney9', 'WebDeveloperCom'):1.0 ('AffiliateMoney9', 'onehunnidt'):1.0 ('Bako_Douche_Bag', 'Kriegsson'):1.0 ('DeafTechNews', 'mosesbillacura'):1.0 ('DeafTechNews', 'rtsuchi'):1.0>
Embeddedness@5(('el_pais', 'ElviraLindo')) = 0.1581302834410741
--> elapsed time: 0:15:12 <--
-------------------------
ClusteringCoefficient = 0.15427656957082322
--> elapsed time: 0:01:26 <--
AvgUserMetric = 0.2732379819086387
--> elapsed time: 0:00:22 <--
Assortativity = -0.07719727387408556
--> elapsed time: 0:00:00 <--
==================================================
Testing graphs/facebook_combined.txt network
4039 users and 88234 contact relationships
User 9 has 57 contacts
-------------------------
UserClusteringCoefficient@5 : <32:1.0 33:1.0 35:1.0 42:1.0 44:1.0>
UserClusteringCoefficient@5(9) = 0.39724310776942356
--> elapsed time: 0:00:02 <--
-------------------------
Embeddedness@5 : <(4, 181):1.0 (4, 275):1.0 (6, 147):1.0 (8, 91):1.0 (8, 259):1.0>
Embeddedness@5((9, 3)) = 0.22033898305084745
--> elapsed time: 0:01:18 <--
-------------------------
ClusteringCoefficient = 0.5191742775433075
--> elapsed time: 0:00:05 <--
AvgUserMetric = 0.6055467186200876
--> elapsed time: 0:00:02 <--
Assortativity = 0.06357722918564918
--> elapsed time: 0:00:00 <--
